In [3]:
#서버 접속부분
!pip install parallel-ssh
!pip install tqdm
!pip install opencv-python
!pip install scp

  Using cached scp-0.13.3-py2.py3-none-any.whl (8.2 kB)


In [4]:
from paramiko import SSHClient
from scp import SCPClient
import paramiko
import time

def send_command():
    ipaddress = "210.115.229.4"
    port="5022"
    id="seeds"
    pw='1111133333'

    ssh= SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ipaddress,port=5022,username=id,password=pw)

    scp= SCPClient(ssh.get_transport())
    
    remote_path ="/home/seeds/iconms12/tmp/image_Captioning/image_ex"
    scp.put('c:/Users/iconms12/realtime_objectdetecting/img/person.png',remote_path) #서버에 전송
    #ssh.exec_command(command)
    stdin, stdout, stderr = ssh.exec_command('cd /home/seeds/iconms12/tmp/image_Captioning;/home/seeds/iconms12/tmp/yes/bin/python run.py person.png' ,get_pty=True)
    for line in iter(stdout.readline, ""):
        print(line, end="")
    scp.get('/home/seeds/iconms12/tmp/image_Captioning/text_output/output.txt','C:/Users/iconms12/realtime_objectdetecting/msg_file')

    #ssh.exec_command('rm -rf ./iconms12/tmp/image_Captioning/image_input')

    scp.close()

In [9]:
send_command()

NameError: name 'send_command' is not defined

In [5]:

import cv2
import numpy as np
import os
# 웹캠 신호 받기
VideoSignal = cv2.VideoCapture(0)
VideoSignal.set(cv2.CAP_PROP_FPS, 15)
# YOLO 가중치 파일과 CFG 파일 로드
YOLO_net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")

# YOLO NETWORK 재구성
classes = []
with open("yolo.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = YOLO_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in YOLO_net.getUnconnectedOutLayers()]
count2=0;
count=1;

while True:
    # 웹캠 프레임
    ret, frame = VideoSignal.read()
    h, w, c = frame.shape

    # YOLO 입력
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0),
    True, crop=False)
    YOLO_net.setInput(blob)
    outs = YOLO_net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:

        for detection in out:

            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * w)
                center_y = int(detection[1] * h)
                dw = int(detection[2] * w)
                dh = int(detection[3] * h)
                # Rectangle coordinate
                x = int(center_x - dw / 2)
                y = int(center_y - dh / 2)
                boxes.append([x, y, dw, dh])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.45, 0.4)

    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            score = confidences[i]

            # 경계상자와 클래스 정보 이미지에 입력
            cv2.rectangle(frame, (x-50, y-50), (x + w+50, y + h+50), (0, 0, 255), 5)
            cv2.putText(frame, label, (x, y - 20), cv2.FONT_ITALIC, 1, 
            (0, 0, 255), 1)
            
            if count % 3 == 0 or label == 'person' : #3프레임 단위로 저장
                count2+=1
                img_trim = frame[y-50 :y +h +50, x -50 :x +w +50]
                img_name = "c:/Users/iconms12/realtime_objectdetecting/img/person.png"
                cv2.imwrite(img_name, img_trim)
                send_command()
                a=open('C:/Users/iconms12/realtime_objectdetecting/msg_file/output.txt','r',encoding='UTF8')
                
                read= a.readline();
                if(read == '<위험상황>'):
                    print(a.read())
                    #문자 -
                    #이메일 -
                else:
                    print(a.read())
                   #os.remove('c:/Users/iconms12/realtime_objectdetecting/img/person_frame.png')
                a.close()
            count+=1
            
            
            
    cv2.imshow("YOLOv3", frame)

    if cv2.waitKey(1) == ord('q'):
        break

=> Loading checkpoint
현재위치: 손민성 집의 CCTV 
 촬영시각: 2020년11월 12일 01시 59분 41초 
 상세내용: 한남자가<UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK>
=> Loading checkpoint
현재위치: 손민성 집의 CCTV 
 촬영시각: 2020년11월 12일 01시 59분 50초 
 상세내용: 한남자가<UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK>
=> Loading checkpoint
현재위치: 손민성 집의 CCTV 
 촬영시각: 2020년11월 12일 01시 59분 58초 
 상세내용: 한남자가<UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK><UNK>
